# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,20.98,81,100,5.96,PN,1731463570
1,1,jinghong,21.9910,100.7341,23.25,74,0,0.67,CN,1731463572
2,2,taulaga,-11.0553,-171.0883,28.77,71,1,7.25,AS,1731463573
3,3,bethel,41.3712,-73.4140,5.12,53,0,5.66,US,1731463574
4,4,gaspe,48.8334,-64.4819,0.12,64,75,8.23,CA,1731463493


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# plot cities on the holoview plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0),:
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,jinghong,21.9910,100.7341,23.25,74,0,0.67,CN,1731463572
201,201,collie,-33.3667,116.1500,24.07,37,0,2.61,AU,1731463807
212,212,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1731463820
229,229,kuchera,26.9839,73.9719,21.16,33,0,1.99,IN,1731463840
243,243,san patricio,28.0170,-97.5169,21.97,64,0,3.36,US,1731463857
277,277,salalah,17.0151,54.0924,22.05,68,0,1.54,OM,1731463896
287,287,tshabong,-26.0500,22.4500,21.76,16,0,1.77,BW,1731463909
355,355,jaisalmer,26.9147,70.9181,23.38,33,0,0.74,IN,1731463988
360,360,ar ruways,24.1103,52.7306,24.66,73,0,2.28,AE,1731463993
371,371,mayahi,13.9553,7.6712,23.96,23,0,3.98,NE,1731464007


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,jinghong,CN,21.9910,100.7341,74,
201,collie,AU,-33.3667,116.1500,37,
212,remire-montjoly,GF,4.9167,-52.2667,94,
229,kuchera,IN,26.9839,73.9719,33,
243,san patricio,US,28.0170,-97.5169,64,
277,salalah,OM,17.0151,54.0924,68,
287,tshabong,BW,-26.0500,22.4500,16,
355,jaisalmer,IN,26.9147,70.9181,33,
360,ar ruways,AE,24.1103,52.7306,73,
371,mayahi,NE,13.9553,7.6712,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)

    # Convert the API response to JSON format
    name_address = name_address.json()
  
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
200
jinghong - nearest hotel: No hotel found
200
collie - nearest hotel: Federal Hotel
200
remire-montjoly - nearest hotel: Complexe Belova
200
kuchera - nearest hotel: No hotel found
200
san patricio - nearest hotel: No hotel found
200
salalah - nearest hotel: Muscat International Hotel
200
tshabong - nearest hotel: No hotel found
200
jaisalmer - nearest hotel: Jeet Mahal
200
ar ruways - nearest hotel: No hotel found
200
mayahi - nearest hotel: No hotel found
200
brownsville - nearest hotel: Cameron Hotel
200
sanford - nearest hotel: Park Place Inn, Cottages, & Catering
200
dickinson - nearest hotel: Days Inn by Wyndham Dickinson TX
200
goz beida - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
1,jinghong,CN,21.9910,100.7341,74,No hotel found
201,collie,AU,-33.3667,116.1500,37,Federal Hotel
212,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
229,kuchera,IN,26.9839,73.9719,33,No hotel found
243,san patricio,US,28.0170,-97.5169,64,No hotel found
277,salalah,OM,17.0151,54.0924,68,Muscat International Hotel
287,tshabong,BW,-26.0500,22.4500,16,No hotel found
355,jaisalmer,IN,26.9147,70.9181,33,Jeet Mahal
360,ar ruways,AE,24.1103,52.7306,73,No hotel found
371,mayahi,NE,13.9553,7.6712,23,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)